In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210703.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,30/06/2021 04:20:35,Umberto D (1952),Dunston Checks In (1996),Black Stallion (1979),BAD PARENTING,IT WAS THE 90'S!,LIVING LIFE A QUARTER MILE AT A TIME,[Default to Nicolas Cage]
1,30/06/2021 05:53:33,Umberto D (1952),Black Stallion (1979),Dunston Checks In (1996),IT WAS THE 90'S!,[Default to Nicolas Cage],BAD PARENTING,LIVING LIFE A QUARTER MILE AT A TIME
2,30/06/2021 14:13:16,Dunston Checks In (1996),Black Stallion (1979),Umberto D (1952),LIVING LIFE A QUARTER MILE AT A TIME,IT WAS THE 90'S!,BAD PARENTING,[Default to Nicolas Cage]
3,30/06/2021 21:02:24,Black Stallion (1979),Umberto D (1952),Dunston Checks In (1996),BAD PARENTING,IT WAS THE 90'S!,LIVING LIFE A QUARTER MILE AT A TIME,[Default to Nicolas Cage]
4,01/07/2021 23:11:48,Black Stallion (1979),Dunston Checks In (1996),Umberto D (1952),IT WAS THE 90'S!,BAD PARENTING,LIVING LIFE A QUARTER MILE AT A TIME,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Umberto D (1952)', 'Dunston Checks In (1996)',
       'Black Stallion (1979)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Umberto D (1952)': <Candidate('Umberto D (1952)')>,
 'Dunston Checks In (1996)': <Candidate('Dunston Checks In (1996)')>,
 'Black Stallion (1979)': <Candidate('Black Stallion (1979)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Umberto D (1952), Dunston Checks In (1996), Black Stallion (1979))>,
 <Ballot(Umberto D (1952), Black Stallion (1979), Dunston Checks In (1996))>,
 <Ballot(Dunston Checks In (1996), Black Stallion (1979), Umberto D (1952))>,
 <Ballot(Black Stallion (1979), Umberto D (1952), Dunston Checks In (1996))>,
 <Ballot(Black Stallion (1979), Dunston Checks In (1996), Umberto D (1952))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                   Votes  Status
------------------------  -------  --------
Black Stallion (1979)           2  Hopeful
Umberto D (1952)                2  Hopeful
Dunston Checks In (1996)        1  Rejected

FINAL RESULT
Candidate                   Votes  Status
------------------------  -------  --------
Black Stallion (1979)           3  Elected
Umberto D (1952)                2  Rejected
Dunston Checks In (1996)        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                               Votes  Status
------------------------------------  -------  --------
IT WAS THE 90'S!                            2  Hopeful
BAD PARENTING                               2  Hopeful
LIVING LIFE A QUARTER MILE AT A TIME        1  Rejected
[Default to Nicolas Cage]                   0  Rejected

FINAL RESULT
Candidate                               Votes  Status
------------------------------------  -------  --------
IT WAS THE 90'S!                            3  Elected
BAD PARENTING                               2  Rejected
LIVING LIFE A QUARTER MILE AT A TIME        0  Rejected
[Default to Nicolas Cage]                   0  Rejected

